In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
train = pd.read_csv('./training set/train_scalied.csv')
test = pd.read_csv('./preprocessed data/new_test_err.csv', parse_dates=['time'])
train_problem = pd.read_csv('./preprocessed data/new_train_problem.csv', parse_dates=['time'])
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

## train-test split 진행 후 결과 확인

In [8]:
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train, problem, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(12000, 70) (3000, 70) (12000,) (3000,)


In [11]:
train_x = x_train
train_y = pd.DataFrame(y_train)
train_y.columns = ['problem']
train = pd.concat([train_x,train_y], axis =1)
train

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.2641,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.33.1095,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.73.2571,fwver_04.82.1730,fwver_05.15.2090,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_5,quality_6,quality_8,quality_11,quality_12,multiple_model,problem
9839,-0.043063,-0.340504,-0.239957,-0.414143,0.416264,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,0.028099,-0.225560,0.052354,-0.052361,-0.037669,0.255652,-0.030391,1.485085,-0.279724,-0.225162,-0.114028,-0.071956,-0.075181,0.070497,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.0,1.0
9680,-0.142438,-0.340504,-0.239957,-0.414143,0.277894,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.274872,-0.107477,-0.052361,-0.037669,-0.064734,-0.069761,-0.652306,-0.279724,-0.225162,-0.114028,-0.071956,-0.075181,-0.193844,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.0,1.0
7093,0.159861,1.505998,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,1.599005,-0.107477,-0.052361,-0.037669,-0.064734,-0.056638,1.210690,-0.005622,-0.225162,-0.114028,-0.071956,-0.075181,1.524370,-0.154519,0.495646,-0.198956,2.046686,-0.037428,0.628662,0.263653,-0.080131,1.241343,3.120791,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.0,0.0
11293,0.100690,1.306790,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.084667,-0.107477,-0.052361,-0.037669,-0.064734,-0.069761,0.762994,-0.279724,-0.225162,-0.114028,0.231975,0.877285,-0.017617,-0.154519,0.339732,-0.198956,0.187291,0.911224,0.628662,-0.114398,-0.080131,1.228969,0.069939,0.201088,-0.023285,-0.072722,-0.050180,-0.060788,-0.044469,-0.043608,-0.06688,0.0,1.0
820,-0.095406,-0.340504,-0.239957,1.044665,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.00816